In [1]:
import numpy as np
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys #模拟键盘输入的
from time import sleep
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import warnings
warnings.filterwarnings('ignore')

In [2]:
option=Options()
option.add_argument("--headless")#无头模式 
wd=webdriver.Chrome(options=option)#隐藏chrome页面，省去页面渲染。
wd.get('https://www.jd.com')

In [4]:
WebDriverWait(wd,10).until(
    ec.presence_of_all_elements_located(
        (By.ID,'key')
    )
)

kw=wd.find_element_by_id('key')
kw.send_keys('鼠标')
kw.send_keys(Keys.RETURN)#回车键'''
sleep(2)
sort_by_sales=wd.find_element_by_xpath('//div[@class="f-sort"]/a[2]')
sort_by_sales.click()

In [5]:
goods_list=wd.find_element_by_id('J_goodsList')
y=goods_list.rect['y']+goods_list.rect['height']
wd.execute_script('window.scrollTo(0,{})'.format(y))#format格式化函数把y放进{}
sleep(4)
skip=wd.find_element_by_xpath('//div[@id="J_bottomPage"]/span[@class="p-skip"]//input[@class="input-txt"]')
skip.send_keys(Keys.COMMAND,'a')
sleep(1)
skip.send_keys('65')#这里可以修改为想要的页数
skip.send_keys(Keys.RETURN)
sleep(2)

爬取失败的说明有一个商品是没有商品评价那个选项的。

In [6]:
row=[]

In [7]:
for j in np.arange(65, 101):
    print("---------Page{}---------".format(j))
    sleep(3)
    WebDriverWait(wd, 3).until(
        ec.presence_of_all_elements_located(
            (By.ID, 'J_goodsList')))
    # 先获取整个商品区域的尺寸坐标
    goods_list = wd.find_element_by_id('J_goodsList')
    y = goods_list.rect['y']+goods_list.rect['height']
    wd.execute_script('window.scrollTo(0,{})'.format(y))  # format格式化函数把y放进{}
    sleep(2)
    WebDriverWait(wd, 3).until(
        ec.presence_of_all_elements_located(
            (By.XPATH, '//div[@class="ml-wrap"]//div[@id="J_goodsList"]')))
    goods = wd.find_elements_by_xpath(
        '//div[@id="J_searchWrap"]//ul[@class="gl-warp clearfix"]/li')
    for i in range(len(goods)):
     #   try:
        # sleep(2)
        try:
            goods[i].click()
        except BaseException:
            continue
        #print('---Item Code {}---'.format(i+1))
        sleep(2)
        try:
            windows = wd.window_handles
            wd.switch_to.window(windows[1])
        except BaseException:
            continue
        # 这个try是用来判断，是不是正常进入到网页端商品界面
        # 如果不小心跳到了移动端，必然没有p-price
        try:
            WebDriverWait(wd, 3).until(
                ec.presence_of_all_elements_located(
                    (By.XPATH, '//div[@class="dd"]/span[@class="p-price"]')))
        except BaseException:
            wd.close()
            wd.switch_to.window(windows[0])
            continue

        shop_name = wd.find_element_by_xpath(
            '//div[@class="w"]//div[@class="name"]').text

        try:
            score1 = wd.find_element_by_xpath(
                '//div[@class="score-parts"]/div[1]/span[@class="score-detail"]').text  # 商品
            score2 = wd.find_element_by_xpath(
                '//div[@class="score-parts"]/div[2]/span[@class="score-detail"]').text  # 物流
            score3 = wd.find_element_by_xpath(
                '//div[@class="score-parts"]/div[3]/span[@class="score-detail"]').text  # 售后
            score = [score1, score2, score3]
        except BaseException:
            score = '自营'

        sku_name = wd.find_element_by_xpath('//div[@class="sku-name"]').text
        price = wd.find_element_by_xpath(
            '//div[@class="dd"]/span[@class="p-price"]').text

        try:
            quan = wd.find_element_by_id(
                'summary-quan').text[6:].replace('\n', ' ')
        except BaseException:
            quan = '无优惠券'

        try:
            cuxiao = wd.find_element_by_xpath('//div[@class="prom-item"]').text
        except BaseException:
            cuxiao = '无促销'

        # 因为运费可能放在倒数第二个或者最后一个div里面，所以这里用一个or|判断
        yunfei = wd.find_element_by_xpath(
            '//div[@class="store clearfix"]/div[last()]|//div[@class="store clearfix"]/div[last()-1]').text.replace('\n', '')

        sleep(1)
        comment_list = wd.find_element_by_xpath(
            '//div[@class="tab-main large"]//li[@data-offset="38"]')
        y1 = comment_list.rect['y']+comment_list.rect['height']
        wd.execute_script('window.scrollTo(0,{})'.format(y1)
                          )  # format格式化函数把y放进{}
        sleep(1)
        # 看看有没有商品评价，没有的话就跳过
        try:
            a = wd.find_element_by_xpath(
                '//div[@class="ETab"]//ul/li[@data-anchor="#comment"]')
            a.click()
        except BaseException:
            sleep(2)
            # continue
            # 这里如果用continue会报错，会被没有评论的卡住进度,且跳过循环内的所有剩下语句
        sleep(4)
        # 这里不再判断评论是否出现，有些没商品评价的可能这里报错就导致程序提前终止
        # WebDriverWait(wd,3).until(
        #    ec.presence_of_all_elements_located(
        #        (By.XPATH,'//div[contains(@class,"small")]/ul/li[@clstag="shangpin|keycount|product|haoping"]')))
        try:
            WebDriverWait(wd, 3).until(
                ec.presence_of_all_elements_located(
                    (By.XPATH, '//div[@class="tab-main small"]//li[@class="comm-curr-sku"]')))
            wd.find_element_by_xpath(
                '//div[@class="tab-main small"]//li[@class="comm-curr-sku"]').click()
        except:
            sleep(1)
        sleep(5)
        try:
            WebDriverWait(wd, 3).until(
                ec.presence_of_all_elements_located(
                    (By.XPATH, '//div[contains(@class,"small")]/ul/li[@clstag="shangpin|keycount|product|shaidantab"]')))
            shaitu = wd.find_element_by_xpath(
                '//div[contains(@class,"small")]/ul/li[@clstag="shangpin|keycount|product|shaidantab"]').text.replace("万", "0000").replace("+", "")
        except BaseException:
            shaitu = '爬取失败'
        try:
            videoshow = wd.find_element_by_xpath(
                '//div[contains(@class,"small")]/ul/li[@clstag="shangpin|keycount|product|pingjiashipin"]').text.replace("万", "0000").replace("+", "")
        except BaseException:
            videoshow = '爬取失败'
        try:
            zhuiping = wd.find_element_by_xpath(
                '//div[contains(@class,"small")]/ul/li[@clstag="shangpin|keycount|product|zhuiping"]').text.replace("万", "0000").replace("+", "")
        except BaseException:
            zhuiping = '爬取失败'
        try:
            haoping = wd.find_element_by_xpath(
                '//div[contains(@class,"small")]/ul/li[@clstag="shangpin|keycount|product|haoping"]').text.replace("万", "0000").replace("+", "")
        except BaseException:
            haoping = '爬取失败'
        try:
            zhongping = wd.find_element_by_xpath(
                '//div[contains(@class,"small")]/ul/li[@clstag="shangpin|keycount|product|zhongping"]').text.replace("万", "0000").replace("+", "")
        except BaseException:
            zhongping = '爬取失败'
        try:
            chaping = wd.find_element_by_xpath(
                '//div[contains(@class,"small")]/ul/li[@clstag="shangpin|keycount|product|chaping"]').text.replace("万", "0000").replace("+", "")
        except BaseException:
            chaping = '爬取失败'
            #wd.close()
        else:
            print("成功", end=",")
        try:
            haopingrate = wd.find_element_by_xpath(
                '//div[@class="comment-percent"]/div[@class="percent-con"]').text
        except BaseException:
            haopingrate = '爬取失败'
        try:
            pingjiabiaoqian = wd.find_element_by_xpath(
                '//div[@class="percent-info"]/div[@class="tag-list tag-available"]').text
        except BaseException:
            pingjiabiaoqian = '爬取失败'

        # 判断是否有视频，如果有则返回‘1’，不是则返回‘0’
        try:
            video = wd.find_element_by_xpath(
                '//div[@class="jqzoom main-img"]//span[@class="video-icon J-video-icon"]')
        except BaseException:
            video = 0
        else:
            video = 1

        data = {
            "店名": shop_name,
            "商品名": sku_name,
            "价格": price,
            "优惠券": quan,
            "促销": cuxiao,
            "运费": yunfei,
            "晒图": shaitu,
            "晒视频": videoshow,
            "追评": zhuiping,
            "好评": haoping,
            "中评": zhongping,
            "差评": chaping,
            "好评率": haopingrate,
            "视频": video,
            "店铺分数(若没有显示为自营)": score,
            "评价标签": pingjiabiaoqian
        }
        row.append(data)

        wd.close()
        wd.switch_to.window(windows[0])
        sleep(1)
    # except BaseException:
        # print("出现问题，进入下一个商品")
        # continue

    sort_next_page = wd.find_element_by_class_name('pn-next')
    sort_next_page.click()

wd.quit()

---------Page65---------
成功,

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=102.0.5005.61)


In [ ]:
len(row)

2124

In [ ]:
table=pd.DataFrame(row)
table.to_excel('鼠标65往后.xlsx',index=False)
table

,店名,商品名,价格,优惠券,促销,运费,晒图,晒视频,追评,好评,中评,差评,好评率,视频,店铺分数(若没有显示为自营),评价标签
0,翼熙外设产品专营店,【无线鼠标】漫威官方联名同款钢铁侠美国队长蚁人电竞游戏鼠标电池款发光滑鼠M无线光电蓝牙鼠标o...,￥ 263.80,,无促销,支持极速审核|闪电退款,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.00 低, 9.19 中, 8.21 低]",爬取失败
1,翼熙外设产品专营店,MACHENIKE机械师M7二代漫威联名同款钢铁侠无线双模鼠标办公游戏电竞宏非蓝牙适用华为o...,￥ 463.80,,无促销,支持极速审核|闪电退款,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.00 低, 9.19 中, 8.21 低]",爬取失败
2,翼熙外设产品专营店,MACHENIKE机械师M200有线鼠标M1/M3电竞游戏吃鸡笔记本电脑办公专用黑色USBo...,￥ 243.80,,无促销,支持极速审核|闪电退款,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.00 低, 9.19 中, 8.21 低]",爬取失败
3,翼熙外设产品专营店,达尔优官方轻量化蓝牙无线双模鼠标电竞游戏电脑笔记本有线办公支持华为小米宏编程RGB充电底座o...,￥ 263.80,,无促销,支持极速审核|闪电退款,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.00 低, 9.19 中, 8.21 低]",爬取失败
4,翼熙外设产品专营店,达尔优官方轻量化蓝牙无线双模鼠标电竞游戏电脑笔记本有线办公支持华为小米宏编程RGB充电底座o...,￥ 383.80,,无促销,支持极速审核|闪电退款,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.00 低, 9.19 中, 8.21 低]",爬取失败
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,蚕颜悦色CYYS旗舰店,MDNG-通用超静音鼠标男女生可爱可充电式游戏办公适用mac苹果华为戴尔惠普联想 M107小...,￥ 71.10,,满减 满200元减10元，满500元减30元，满1000元减80元 详情>>,支持免费上门取退|30天价保|极速审核|退换货运费险,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.69 低, 9.19 中, 8.96 中]",爬取失败
2120,蚕颜悦色CYYS旗舰店,MDNG-通用超静音鼠标男女生可爱可充电式游戏办公适用mac苹果华为戴尔惠普联想 M107白...,￥ 88.20,,满减 满200元减10元，满500元减30元，满1000元减80元 详情>>,支持免费上门取退|30天价保|极速审核|退换货运费险,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.69 低, 9.19 中, 8.96 中]",爬取失败
2121,蚕颜悦色CYYS旗舰店,MDNG-通用超静音鼠标/USB双模式兼容笔记本台式电脑小巧便携质感好 灰色全新盒装 自备电...,￥ 49.50,,满减 满200元减10元，满500元减30元，满1000元减80元 详情>>,支持免费上门取退|30天价保|极速审核|退换货运费险,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.69 低, 9.19 中, 8.96 中]",爬取失败
2122,蚕颜悦色CYYS旗舰店,MDNG-通用超静音鼠标女生可爱笔记本电脑办公颜值无限 粉色（蓝牙+无线）双模版 官方标配,￥ 171.00,,满减 满200元减10元，满500元减30元，满1000元减80元 详情>>,支持免费上门取退|30天价保|极速审核|退换货运费险,晒图(0),视频晒单(0),追评(0),好评(0),中评(0),差评(0),100%,0,"[8.69 低, 9.19 中, 8.96 中]",爬取失败
